In [20]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import keras.backend as K
import pandas as pd
import os
import math

from keras.models import load_model
from sklearn.metrics import r2_score
from dataset import load_dataset, polyfit, expfit, shuffle_apply_scaler, perm_distribution, shuffle_apply_scaler_testing
from model import MSE_function, huber_fn, MSE_model, huber_model
from keras.optimizers import Adam
from keras.layers import LSTM, Dense, Dropout
from keras.metrics import RootMeanSquaredError, MeanAbsolutePercentageError, MeanAbsoluteError
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [2]:
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9



In [3]:
df, perm_df, param_df, time_df = load_dataset('../dataset/CombinedDataset.xlsx', '1')
means, Distribution = perm_distribution(df)
X1_arr_train_shuffle, X2_arr_train_shuffle, Y_arr_train_shuffle, X1_arr_valid_shuffle, X2_arr_valid_shuffle, Y_arr_valid_shuffle, X1_arr_test_shuffle, X2_arr_test_shuffle, Y_arr_test_shuffle, timestamp_test = shuffle_apply_scaler(df, perm_df, param_df, time_df)

df_testing, perm_df_testing, param_df_testing, time_df_testing = load_dataset('../dataset/CombinedDataset_testing.xlsx','1')
X1_arr_shuffle_testing, X2_arr_shuffle_testing, Y_arr_shuffle_testing = shuffle_apply_scaler_testing(df,df_testing,perm_df_testing,param_df_testing)

c:\Users\admin\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\admin\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\admin\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [4]:
offset = 1.0
ic_pt = means[330]
multiplier = 5.0

y_max = np.max(Distribution[0]).astype('float32')
y_min = np.min(Distribution[0]).astype('float32')

p_list = polyfit(df)
a_fit, b_fit, c_fit = expfit(df)

def custom_loss_function(y_true, y_pred):
    delta = 5
    diff = y_true - y_pred
    is_small_error = tf.abs(diff) < delta
    squared_loss = tf.square(diff) / 2
    linear_loss  = tf.multiply(tf.abs(diff), delta) - 0.5 * delta**2
    error = tf.where(is_small_error, squared_loss, linear_loss)
    is_small_value = y_true < ic_pt
    polynomial_term = tf.math.polyval(p_list, y_true)
    a_fit_float32 = tf.cast(a_fit, dtype=tf.float32)
    b_fit_float32 = tf.cast(b_fit, dtype=tf.float32)
    c_fit_float32 = tf.cast(c_fit, dtype=tf.float32)
    exp_term = tf.add(tf.multiply(a_fit_float32, tf.exp(tf.multiply(-1*b_fit_float32, y_true))), c_fit_float32)
    clf_coe = tf.where(is_small_value, polynomial_term, exp_term)
    clf_coe_reversed = tf.add(offset,tf.divide(tf.subtract(y_max, clf_coe), tf.subtract(y_max, y_min)))
    return K.mean(tf.multiply(tf.multiply(multiplier, clf_coe_reversed), error))


In [15]:
file_name = '../models/R168_Dropout.1_bs1024_lr0.005_Delta5_CLFHuber5_Adam'
filepath=os.path.join(file_name + '.h5')
regressor=load_model(filepath,
                     custom_objects={'custom_loss_function': custom_loss_function})

# file_name = '../models/model_repetitions/R1616_Dropout.1_bs1024_lr0.005_Huber5_softmax0'
# filepath=os.path.join(file_name + '.h5')
# regressor=load_model(filepath,
#                      custom_objects={'huber_fn': huber_fn})

# file_name = '../models/model_repetitions/R168_Dropout.1_bs1024_lr0.005_MSE0'
# filepath=os.path.join(file_name + '.h5')
# regressor=load_model(filepath,
#                      custom_objects={'MSE_function': MSE_function})

In [16]:
Perm_Max=max(df['Permeability'])
Perm_Min=min(df['Permeability'])
Perm_Max, Perm_Min

def postprocess_perm(arr):
    arr=arr*(Perm_Max-Perm_Min)+Perm_Min
    return arr


In [17]:
#performance on testing set 1
predictions = regressor.predict([X1_arr_test_shuffle, X2_arr_test_shuffle])

predictions_flat = postprocess_perm(predictions.flatten())
Y_arr_test_shuffle_flat = postprocess_perm(Y_arr_test_shuffle.flatten())

mape = abs((predictions_flat-Y_arr_test_shuffle_flat)/Y_arr_test_shuffle_flat)
r2 = r2_score(Y_arr_test_shuffle_flat, predictions_flat)
mse = ((predictions_flat - Y_arr_test_shuffle_flat)**2)
mae = abs(predictions_flat - Y_arr_test_shuffle_flat)

print("MAPE: "+str(mape.mean()))
print("r2: "+str(r2))
print("MAE: "+str(mae.mean()))
print("MSE: "+str(mse.mean()))
print("RMSE: "+str(math.sqrt(mse.mean())))

1114/1114 [==============================] - 33s 29ms/step
MAPE: 0.06750317112143457
r2: 0.8658438238419849
MAE: 3.9176192055417003
MSE: 33.675088792149005
RMSE: 5.803024107493351


In [18]:
# performance on testing set 2
predictions = regressor.predict([X1_arr_shuffle_testing, X2_arr_shuffle_testing])

predictions_flat = postprocess_perm(predictions.flatten())
Y_arr_test_shuffle_flat = postprocess_perm(Y_arr_shuffle_testing.flatten())

mape = abs((predictions_flat-Y_arr_test_shuffle_flat)/Y_arr_test_shuffle_flat)
r2 = r2_score(Y_arr_test_shuffle_flat, predictions_flat)
mse=((predictions_flat-Y_arr_test_shuffle_flat)**2)
mae=abs(predictions_flat-Y_arr_test_shuffle_flat)

print("MAPE: "+str(mape.mean()))
print("r2: "+str(r2))
print("MAE: "+ str(mae.mean()))
print("MSE: "+str(mse.mean()))
print("RMSE: "+str(math.sqrt(mse.mean())))

643/643 [==============================] - 18s 29ms/step
MAPE: 0.06384467457902218
r2: 0.8490514170216203
MAE: 3.2596176645229367
MSE: 26.20543907658134
RMSE: 5.119124835026134


In [28]:
results_df = pd.DataFrame(columns=['MAPE_set1', 'R2_set1', 'MAE_set1', 'MSE_set1', 'RMSE_set1', 'MAPE_set2', 'R2_set2', 'MAE_set2', 'MSE_set2', 'RMSE_set2'])


In [21]:
def pred_model(index):
    folder_name = '../models/model_repetitions/'

    file_name = 'R168_Dropout.1_bs1024_lr0.005_Delta5_CLFHuber5_Adam' + str(index)
    # file_name = 'R1616_Dropout.1_bs1024_lr0.005_Huber5_softmax' + str(index)
    # file_name = 'R168_Dropout.1_bs1024_lr0.005_MSE' + str(index)
    
    filepath=os.path.join(folder_name + file_name + '.h5')
    regressor=load_model(filepath,
                         custom_objects={'custom_loss_function': custom_loss_function})
    # regressor=load_model(filepath,
    #                      custom_objects={'huber_fn': huber_fn})
    # regressor=load_model(filepath,
    #                      custom_objects={'MSE_function': MSE_function})
    

    print('This is set #' + str(index))
    set1_metrics = set1_pred(regressor)
    set2_metrics = set2_pred(regressor)

    # Append the results to the DataFrame
    global results_df
    new_row = pd.DataFrame([{
        **set1_metrics,
        **set2_metrics
    }])
    results_df = pd.concat([results_df, new_row], ignore_index=True)

def set1_pred(regressor):
    predictions = regressor.predict([X1_arr_test_shuffle, X2_arr_test_shuffle])
    
    predictions_flat = postprocess_perm(predictions.flatten())
    Y_arr_test_shuffle_flat = postprocess_perm(Y_arr_test_shuffle.flatten())
    
    #mape
    mape = abs((predictions_flat-Y_arr_test_shuffle_flat)/Y_arr_test_shuffle_flat)
    r2 = r2_score(Y_arr_test_shuffle_flat, predictions_flat)
    mse = ((predictions_flat - Y_arr_test_shuffle_flat)**2)
    mae = abs(predictions_flat - Y_arr_test_shuffle_flat)
    
    print("MAPE: "+str(mape.mean()))
    print("r2: "+str(r2))
    print("MAE: "+str(mae.mean()))
    print("MSE: "+str(mse.mean()))
    print("RMSE: "+str(math.sqrt(mse.mean())))
    return {
        'MAPE_set1': mape.mean(),
        'R2_set1': r2,
        'MAE_set1': mae.mean(),
        'MSE_set1': mse.mean(),
        'RMSE_set1': math.sqrt(mse.mean())
    }


def set2_pred(regressor):
    predictions = regressor.predict([X1_arr_shuffle_testing, X2_arr_shuffle_testing])
    
    predictions_flat = postprocess_perm(predictions.flatten())
    Y_arr_test_shuffle_flat = postprocess_perm(Y_arr_shuffle_testing.flatten())
    
    #mape
    mape = abs((predictions_flat-Y_arr_test_shuffle_flat)/Y_arr_test_shuffle_flat)
    from sklearn.metrics import r2_score
    import math
    r2 = r2_score(Y_arr_test_shuffle_flat, predictions_flat)
    mse=((predictions_flat-Y_arr_test_shuffle_flat)**2)
    mae=abs(predictions_flat-Y_arr_test_shuffle_flat)
    
    print("MAPE: "+str(mape.mean()))
    print("r2: "+str(r2))
    print("MAE: "+ str(mae.mean()))
    print("MSE: "+str(mse.mean()))
    print("RMSE: "+str(math.sqrt(mse.mean())))
    print("\n")
    return {
        'MAPE_set2': mape.mean(),
        'R2_set2': r2,
        'MAE_set2': mae.mean(),
        'MSE_set2': mse.mean(),
        'RMSE_set2': math.sqrt(mse.mean())
    }

In [31]:
results_df = pd.DataFrame(columns=['MAPE_set1', 'R2_set1', 'MAE_set1', 'MSE_set1', 'RMSE_set1', 'MAPE_set2', 'R2_set2', 'MAE_set2', 'MSE_set2', 'RMSE_set2'])
for index in range(40):
    pred_model(index)
results_df    
# results_df.to_excel('results.xlsx', index=False, header=True)

This is set #0
1114/1114 [==============================] - 32s 29ms/step
MAPE: 0.06779824340170519
r2: 0.8614164485485235
MAE: 3.8789280355611657
MSE: 34.78642231694975
RMSE: 5.898001552810049
643/643 [==============================] - 18s 28ms/step
MAPE: 0.0655269721674943
r2: 0.8613541594606988
MAE: 3.3028927250979496
MSE: 24.069620633632987
RMSE: 4.906079966086263




C:\Users\admin\AppData\Local\Temp\ipykernel_19076\2922273650.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


This is set #1
1114/1114 [==============================] - 32s 29ms/step
MAPE: 0.06639732528329675
r2: 0.8566509646905189
MAE: 3.953980609284338
MSE: 35.98262585115635
RMSE: 5.998551979532756
643/643 [==============================] - 18s 28ms/step
MAPE: 0.06534999534908557
r2: 0.8649696596405594
MAE: 3.370397162768772
MSE: 23.441951477518586
RMSE: 4.841688907552672




In [32]:
results_df

,MAPE_set1,R2_set1,MAE_set1,MSE_set1,RMSE_set1,MAPE_set2,R2_set2,MAE_set2,MSE_set2,RMSE_set2
0,0.067798,0.861416,3.878928,34.786422,5.898002,0.065527,0.861354,3.302893,24.069621,4.906080
1,0.066397,0.856651,3.953981,35.982626,5.998552,0.065350,0.864970,3.370397,23.441951,4.841689
